In [958]:
import pandas as pd
from sklearn import (svm, preprocessing)
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import (recall_score, precision_score, accuracy_score, confusion_matrix,)
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

In [959]:
data_train = pd.read_csv("./Data/Accident_train.csv")
data_test = pd.read_csv("./Data/Accident_test.csv")
#data_train.head()
#data_test.head()
#data_train.info()
#data_test.info()
#data_train.var()

In [960]:
#data_train["Weekday_of_Collision"].value_counts()
#data_train["Policing_Area"].value_counts()
#data_test["Policing_Area"].value_counts()
#data_test["Collision_Severity"].value_counts()
#data_test["Weekday_of_Collision"].value_counts()
data_train["Ped_Crossing_HC"].value_counts()
#print(type(data_train))

1.0    8529
3.0      26
2.0      18
Name: Ped_Crossing_HC, dtype: int64

In [964]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# lable encoder for data_train
le.fit(data_train['Weekday_of_Collision'])
data_train['Weekday_of_Collision'] = le.transform(data_train['Weekday_of_Collision'])
le.fit(data_train['Policing_Area'].astype(str))
data_train['Policing_Area'] = le.transform(data_train['Policing_Area'].astype(str))

In [965]:
# Converting Nan Values to mean/mode etc.,
#data_train.isna().any()
#data_train.mean()
#data_train.isna().any()
data_train = data_train.fillna(data_train.median())
# Taking care of missing values with most frequent values
#data_train = data_train.apply(lambda x:x.fillna(x.value_counts().index[0]))

In [966]:
# lable encoder for data_test
le.fit(data_test['Collision_Severity'])
data_test['Collision_Severity'] = le.transform(data_test['Collision_Severity'])
le.fit(data_test['Weekday_of_Collision'])
data_test['Weekday_of_Collision'] = le.transform(data_test['Weekday_of_Collision'])
le.fit(data_test['Policing_Area'].astype(str))
data_test['Policing_Area'] = le.transform(data_test['Policing_Area'].astype(str))

In [967]:
#data_train.info()

In [968]:
# Converting Nan Values to mean/mode etc.,
#data_test.isna().any()
#data_test.mean()
data_test = data_test.fillna(data_test.median())
#data_train.isna().any()
#data_test = data_test.apply(lambda x:x.fillna(x.value_counts().index[0]))

In [969]:
#data_train.head()

In [970]:
labels_train = data_train["Collision_Severity"]
data_train_x = data_train.drop(["Collision_Severity", "Collision_Ref_No"], axis=1)

data_test_x = data_test.drop(["Collision_Severity", "Collision_Ref_No"], axis=1)

In [971]:
data_train_x.shape

(8849, 15)

In [972]:
import collections
import numpy as np
from sklearn.decomposition import PCA

# splitting test vales into parts to find get lables
data_variables = train_test_split(data_train_x, labels_train, test_size = 0.25, random_state = 42)
#data_variables = train_test_split(data_train, labels_train, test_size=0.33, random_state=99)
data_train_l, data_test_l, labels_train_l, labels_test_l = data_variables

scaler = preprocessing.StandardScaler().fit(data_train_l)
train_data_scaled = scaler.transform(data_train_l)
test_data_scaled = scaler.transform(data_test_l)

test_data_scaled_x = scaler.transform(data_test_x)

#pca = PCA(n_components = 14)
#train_data_scaled = pca.fit_transform(data_train_l)
#test_data_scaled = pca.transform(data_test_l)
#explained_variance = pca.explained_variance_ratio_

#test_data_scaled_x = pca.transform(data_test_x)

In [973]:
#XGBClassifier?

In [975]:
#from sklearn.ensemble import RandomForestRegressor
#rf = RandomForestRegressor(n_estimators = 1000, random_state = 4)
                    
classifier = XGBClassifier()
classifier.fit(train_data_scaled, labels_train_l)

classifier.fit(train_data_scaled, labels_train_l)

# Predicting test data with train data using classifier
predict_y = classifier.predict(test_data_scaled)

#print(type(predict_y))
a = np.array(predict_y)
print(collections.Counter(a))
#for x in predict_y:
#  print(x)

accuracy = classifier.score(test_data_scaled, labels_test_l)

precision = precision_score(labels_test_l, predict_y, average='micro')
recall = recall_score(labels_test_l, predict_y, average='micro')

cmatrix = confusion_matrix(labels_test_l, predict_y)
print("accuracy  : {}".format(accuracy))
print("accuracy[round]  : {}".format(round(accuracy, 3)))
print("precision : {}".format(precision))
print("recall    : {} \n".format(recall))
print("Confusion matrix \n{}\n\n".format(cmatrix))

Counter({3: 2169, 2: 43, 1: 1})
accuracy  : 0.9023949389968369
accuracy[round]  : 0.902
precision : 0.9023949389968369
recall    : 0.9023949389968369 

Confusion matrix 
[[   0    6   21]
 [   1   27  178]
 [   0   10 1970]]




/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [976]:
#SVC
svc = SVC()
svc.fit(train_data_scaled, labels_train_l)
Y_pred = svc.predict(test_data_scaled)
a = np.array(Y_pred)
predict_svc = collections.Counter(a)
acc_svc = round(svc.score(test_data_scaled, labels_test_l) * 100, 2)
acc_svc

#KNN
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(train_data_scaled, labels_train_l)
Y_pred = knn.predict(test_data_scaled)
a = np.array(Y_pred)
predict_knn = collections.Counter(a)
acc_knn = round(knn.score(test_data_scaled, labels_test_l) * 100, 2)
acc_knn

# Logistic Regression
logreg = LogisticRegression()
logreg.fit(train_data_scaled, labels_train_l)
Y_pred = logreg.predict(test_data_scaled)
a = np.array(Y_pred)
predict_log = collections.Counter(a)
acc_log = round(logreg.score(test_data_scaled, labels_test_l) * 100, 2)
acc_log

# Gaussian Naive Bayes
gaussian = GaussianNB()
gaussian.fit(train_data_scaled, labels_train_l)
Y_pred = gaussian.predict(test_data_scaled)
a = np.array(Y_pred)
predict_gaussian = collections.Counter(a)
acc_gaussian = round(gaussian.score(test_data_scaled, labels_test_l) * 100, 2)
acc_gaussian

# Perceptron
perceptron = Perceptron()
perceptron.fit(train_data_scaled, labels_train_l)
Y_pred = perceptron.predict(test_data_scaled)
a = np.array(Y_pred)
predict_perceptron = collections.Counter(a)
acc_perceptron = round(perceptron.score(test_data_scaled, labels_test_l) * 100, 2)
acc_perceptron

# Linear SVC
linear_svc = LinearSVC()
linear_svc.fit(train_data_scaled, labels_train_l)
Y_pred = linear_svc.predict(test_data_scaled)
a = np.array(Y_pred)
predict_linear_svc = collections.Counter(a)
acc_linear_svc = round(linear_svc.score(test_data_scaled, labels_test_l) * 100, 2)
acc_linear_svc

# Stochastic Gradient Descent
sgd = SGDClassifier()
sgd.fit(train_data_scaled, labels_train_l)
Y_pred = sgd.predict(test_data_scaled)
a = np.array(Y_pred)
predict_sgd = collections.Counter(a)
acc_sgd = round(sgd.score(test_data_scaled, labels_test_l) * 100, 2)
acc_sgd

# Decision Tree
decision_tree = DecisionTreeClassifier()
decision_tree.fit(train_data_scaled, labels_train_l)
Y_pred = decision_tree.predict(test_data_scaled)
a = np.array(Y_pred)
predict_decision_tree = collections.Counter(a)
acc_decision_tree = round(decision_tree.score(test_data_scaled, labels_test_l) * 100, 2)
acc_decision_tree

# Random Forest
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(train_data_scaled, labels_train_l)
Y_pred = random_forest.predict(test_data_scaled)
a = np.array(Y_pred)
predict_random_forest = collections.Counter(a)
acc_random_forest = round(random_forest.score(test_data_scaled, labels_test_l) * 100, 2)
acc_random_forest

# XG Boost
xgboost = XGBClassifier()
xgboost.fit(train_data_scaled, labels_train_l)
Y_pred = xgboost.predict(test_data_scaled)
a = np.array(Y_pred)
predict_xgboost = collections.Counter(a)
acc_xgboost = round(xgboost.score(test_data_scaled, labels_test_l) * 100, 2)
acc_xgboost

models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree', 'X Boost'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree, acc_xgboost],
    'Count': [predict_svc, predict_knn, predict_log, 
              predict_random_forest, predict_gaussian, predict_perceptron, 
              predict_sgd, predict_linear_svc, predict_decision_tree, predict_xgboost]})
models.sort_values(by='Score', ascending=False)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/prepr

,Model,Score,Count
9,X Boost,90.24,"{3: 2169, 2: 43, 1: 1}"
3,Random Forest,89.70,"{3: 2185, 2: 28}"
0,Support Vector Machines,89.47,{3: 2213}
2,Logistic Regression,89.47,{3: 2213}
7,Linear SVC,89.47,{3: 2213}
6,Stochastic Gradient Decent,88.39,"{3: 2183, 1: 24, 2: 6}"
1,KNN,87.08,"{3: 2099, 2: 86, 1: 28}"
8,Decision Tree,85.59,"{3: 1938, 2: 242, 1: 33}"
5,Perceptron,81.11,"{3: 1943, 2: 270}"
4,Naive Bayes,10.12,"{1: 1974, 3: 200, 2: 39}"
